# **March Madness**

<img src="https://a.espncdn.com/photo/2025/0316/2025_ncaam_bracket_filled_3x2.jpg" />

In [2]:
import pandas as pd

In [8]:
df = pd.read_csv("/Users/jonathan/cmse401/Project/CMSE401-Project/data/cbb24.csv")

In [9]:
df

,RK,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG%,EFGD%,...,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,SEED
0,1,Houston,B12,34,30,119.2,85.5,0.9785,49.7,44.0,...,30.2,29.9,39.0,48.4,43.4,34.7,30.0,63.3,10.6,1.0
1,2,Connecticut,BE,34,31,127.1,93.6,0.9712,57.1,45.1,...,26.8,33.3,32.5,58.5,43.7,36.7,31.9,64.6,11.3,1.0
2,3,Purdue,B10,33,29,126.2,94.7,0.9644,56.0,47.7,...,24.7,42.8,23.0,53.2,48.1,40.8,31.4,67.6,11.0,1.0
3,4,Iowa St.,B12,34,27,113.6,86.5,0.9583,51.9,47.1,...,29.4,36.1,35.2,51.7,46.9,34.9,31.5,67.6,6.9,2.0
4,5,Auburn,SEC,34,27,120.7,92.1,0.9573,54.1,43.4,...,30.3,38.2,41.0,54.9,42.8,35.2,29.8,69.8,5.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,358,Stonehill,NEC,30,4,90.3,114.2,0.0629,46.7,52.7,...,31.0,22.6,29.4,47.9,51.7,30.1,35.9,68.1,-22.0,NaN
358,359,St. Francis PA,NEC,28,8,93.1,118.1,0.0608,47.2,53.0,...,31.3,32.6,35.4,45.7,52.9,33.2,35.4,65.4,-18.6,NaN
359,360,IUPUI,Horz,29,6,92.1,116.9,0.0607,46.5,58.2,...,35.5,33.2,33.4,49.5,59.0,24.7,38.0,67.3,-21.6,NaN
360,361,Coppin St.,MEAC,29,2,85.1,111.1,0.0445,42.1,51.3,...,38.6,31.1,38.3,41.9,51.0,28.3,34.5,66.3,-22.9,NaN


**Columns**
 - CONF: Conference
 - G: Games played
 - W: Number of wins
 - ADJOE: Adjusted offensive efficiency
 - ADJDE: Adjusted defensive efficiency
 - BARTHAG: Power rating
 - EFG%: Effective field goal percentage
 - EFGD%: Effective field goal percentage allowed
 - TOR: Turnover rate
 - TORD: Forced turnover rate
 - ORB: Offensive rebound rate
 - DRB: Defensive rebound rate
 - FTR: Free throw rate
 - FTRD: Free throw rate allowed
 - 2P_O: Two-point shooting percentage
 - 2P_D: Two-point shooting percentage allowed
 - 3P_O: Three-point shooting percentage
 - 3P_D: Three-point shooting percentage allowed
 - ADJ_T: Adjusted tempo
 - WAB: Wins above bubble
 - Seed: Seed in NCAA tournament
 

In [7]:
df.columns

Index(['RK', 'TEAM', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG%',
       'EFGD%', 'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D',
       '3P_O', '3P_D', 'ADJ_T', 'WAB', 'SEED'],
      dtype='object')